In [ ]:
#RDDMS demo via Rest-APIs 
#documented in the SWAGGER pages
#http://10.0.1.4:8003/Reservoir/v2#/Resources/

In [ ]:
#Libs needed
!pip install requests
!pip install pandas
!pip install --upgrade matplotlib

In [ ]:
import requests
import json
import urllib.parse
import pandas as pd
import numpy as np

In [ ]:
# Copy your az bearer token to variable bearer_key: PS F:\> az account get-access-token 

bearer_key = ''

RESERVOIR_DDMS_HOST = 'http://osdubootcamp.azure-api.net/reservoir/v2'

#headers = {
#    'accept': 'application/json',
#}
#response = requests.get(RESERVOIR_DDMS_HOST+'/auth/token', headers=headers)
#response.json()['token']


In [ ]:
#List the available data space(s)

In [ ]:
headers = {
    'accept': 'application/json',
    'Authorization': f'Bearer {bearer_key}'
}

params = {
    '$skip': '0',
    '$top': '30',
}

response = requests.get(RESERVOIR_DDMS_HOST+'/dataspaces', params=params, headers=headers)

In [ ]:
print(*response, sep='\n')
print('-----------------')

SelectedDataspace = 0

#You need to URL encode the namespace
print('Dataspace name:', response.json()[SelectedDataspace]['path'])
dataspace_name = urllib.parse.quote(response.json()[SelectedDataspace]['path'], safe="")
print('URL encoded namespace:',dataspace_name)

In [ ]:
headers = {
    'accept': 'application/json',
    'Authorization': f'Bearer {bearer_key}'
}

response = requests.get(f'{RESERVOIR_DDMS_HOST}/dataspaces/{dataspace_name}/resources', headers=headers)

In [ ]:
#print(json.dumps(response.json(),indent=4))
response_flat = pd.json_normalize(response.json()).to_dict(orient='records')
response_df = pd.DataFrame.from_dict(response_flat)
print(response_df)

In [ ]:
# List all object given an object type

In [83]:
response = requests.get(
    f'{RESERVOIR_DDMS_HOST}/dataspaces/{dataspace_name}/resources/resqml20.obj_Grid2dRepresentation',
    params=params,
    headers=headers,
)

In [84]:
print(json.dumps(response.json(),indent=4))

[
    {
        "uri": "eml:///dataspace('demo/bootcamp')/resqml20.obj_Grid2dRepresentation(09939a77-2e2e-4625-bcf5-3d17c9c97ae9)",
        "alternateUris": [],
        "name": "DepthExtractedHorizon",
        "lastChanged": "2023-05-23T12:30:00.000Z",
        "storeLastWrite": "2023-06-02T11:15:43.002Z",
        "storeCreated": "2023-06-02T11:15:43.002Z",
        "activeStatus": "Active",
        "customData": {}
    },
    {
        "uri": "eml:///dataspace('demo/bootcamp')/resqml20.obj_Grid2dRepresentation(17ce9a49-e29e-4463-a572-f84f5a9253ae)",
        "alternateUris": [],
        "name": "DepthExtractedHorizon_HUM_Ref_DC",
        "lastChanged": "2023-05-23T12:30:00.000Z",
        "storeLastWrite": "2023-06-02T11:15:43.002Z",
        "storeCreated": "2023-06-02T11:15:43.002Z",
        "activeStatus": "Active",
        "customData": {}
    },
    {
        "uri": "eml:///dataspace('demo/bootcamp')/resqml20.obj_Grid2dRepresentation(20793438-9ea9-45b8-94a1-07a7fa66102d)",
        "al

In [92]:
SelectedSurface =  0 # first object in dataspace object list
uuid = urllib.parse.quote(response.json()[SelectedSurface]['uri'].split('(')[-1].replace(')','')) 

print('uuid:', uuid)

uuid: 09939a77-2e2e-4625-bcf5-3d17c9c97ae9


In [93]:
print(json.dumps(response.json(),indent=4))
response_flat = pd.json_normalize(response.json()).to_dict(orient='records')
response_df = pd.DataFrame.from_dict(response_flat)
print(response_df)

[
    {
        "uri": "eml:///dataspace('demo/bootcamp')/resqml20.obj_Grid2dRepresentation(09939a77-2e2e-4625-bcf5-3d17c9c97ae9)",
        "alternateUris": [],
        "name": "DepthExtractedHorizon",
        "lastChanged": "2023-05-23T12:30:00.000Z",
        "storeLastWrite": "2023-06-02T11:15:43.002Z",
        "storeCreated": "2023-06-02T11:15:43.002Z",
        "activeStatus": "Active",
        "customData": {}
    },
    {
        "uri": "eml:///dataspace('demo/bootcamp')/resqml20.obj_Grid2dRepresentation(17ce9a49-e29e-4463-a572-f84f5a9253ae)",
        "alternateUris": [],
        "name": "DepthExtractedHorizon_HUM_Ref_DC",
        "lastChanged": "2023-05-23T12:30:00.000Z",
        "storeLastWrite": "2023-06-02T11:15:43.002Z",
        "storeCreated": "2023-06-02T11:15:43.002Z",
        "activeStatus": "Active",
        "customData": {}
    },
    {
        "uri": "eml:///dataspace('demo/bootcamp')/resqml20.obj_Grid2dRepresentation(20793438-9ea9-45b8-94a1-07a7fa66102d)",
        "al

In [97]:
# The following gets the content in json of the XML document containing the metadata for a surface 2d grid.
# Note: the DepthExtractedHorizon depth map represents a SHETLAND_GP._Top Interpretation of the unique SHETLAND_GP._Top Feature.  
# The geometry is specified using Vectors defined using 3d points, and the node number Count and Spacing Value.
# The actual depth data (or peroperties) are referenced through UUID 

In [94]:
headers = {
    'accept': 'application/x-resqml+xml',
    'Authorization': f'Bearer {bearer_key}'
}

params = {
    '$format': 'json',
    'arrayMetadata': 'false',
    'arrayValues': 'false',
    'referencedContent': 'true',
}

response = requests.get(
    f'{RESERVOIR_DDMS_HOST}/dataspaces/{dataspace_name}/resources/resqml20.obj_Grid2dRepresentation/{uuid}',
    params=params,
    headers=headers,
)
print(json.dumps(response.json(),indent=4))

[
    {
        "SchemaVersion": "2.0",
        "Uuid": "09939a77-2e2e-4625-bcf5-3d17c9c97ae9",
        "Citation": {
            "Title": "DepthExtractedHorizon",
            "Originator": "MAAP",
            "Creation": "2023-05-23T12:30:00.000Z",
            "Format": "PDGM-DX ETP Client 2.0.1",
            "$type": "eml20.Citation"
        },
        "ExtraMetadata": [
            {
                "Name": "pdgm/dx/resqml/creatorGroup",
                "Value": "MAAP",
                "$type": "resqml20.NameValuePair"
            },
            {
                "Name": "pdgm/dx/resqml/project",
                "Value": "1ad6a149-c82d-432e-8f8e-0d4f3b7280af",
                "$type": "resqml20.NameValuePair"
            }
        ],
        "RepresentedInterpretation": {
            "ContentType": "application/x-resqml+xml;version=2.0;type=obj_HorizonInterpretation",
            "Title": "SHETLAND_GP._Top",
            "UUID": "ab7cff04-2a41-4047-b604-4af7155c08a8",
            "Uu

In [95]:
headers = {
    'accept': 'application/x-resqml+xml',
    'Authorization': f'Bearer {bearer_key}'
}

params = {
    '$format': 'json',
    'arrayMetadata': 'false',
    'arrayValues': 'false',
    'referencedContent': 'true',
}


response = requests.get(
    f'{RESERVOIR_DDMS_HOST}/dataspaces/{dataspace_name}/resources/resqml20.obj_Grid2dRepresentation/{uuid}/arrays',
    params=params,
    headers=headers,
)
print(json.dumps(response.json(),indent=4))

[
    {
        "uid": {
            "uri": "eml:///dataspace('demo/bootcamp')/eml20.obj_EpcExternalPartReference(6529af45-8c16-4abe-8361-ca135cea89a8)",
            "pathInResource": "/RESQML/09939a77-2e2e-4625-bcf5-3d17c9c97ae9/points_patch0"
        },
        "dimensions": [
            112,
            145
        ],
        "arrayType": "Int8Array",
        "preferredSubarrayDimensions": [],
        "storeLastWrite": "1970-01-01T00:00:00.000Z",
        "storeCreated": "1970-01-01T00:00:00.000Z",
        "customData": {}
    }
]


In [ ]:
uuid_url = urllib.parse.quote(response.json()[0]['uid']['pathInResource'], safe="")
print('uuid_url:', uuid_url)

In [ ]:
headers = {
    'accept': 'application/json',
    'Authorization': f'Bearer {bearer_key}'
    }

params = {
    'format': 'json'
}

response = requests.get(
    f'{RESERVOIR_DDMS_HOST}/dataspaces/{dataspace_name}/resources/resqml20.obj_Grid2dRepresentation/{uuid}/arrays/{uuid_url}',
    params=params,
    headers=headers,
)

In [ ]:
print(response.text)

In [ ]:
print('Data length:',len(response.json()['data']['data']))
print('Dimensions:',response.json()['data']['dimensions'])
assert len(response.json()['data']['data']) == ( int(response.json()['data']['dimensions'][0]) * int(response.json()['data']['dimensions'][1]) )

In [ ]:
xy_array = np.reshape(response.json()['data']['data'], (int(response.json()['data']['dimensions'][0]), int(response.json()['data']['dimensions'][1]))).T

In [ ]:
len(xy_array[:,:])

In [ ]:
z = np.array(response.json()['data']['data'], dtype=np.float32)
z_array = np.reshape(z, (int(response.json()['data']['dimensions'][0]), int(response.json()['data']['dimensions'][1])))
print(xy_array.shape)

In [ ]:
#https://matplotlib.org/stable/gallery/mplot3d/custom_shaded_3d_surface.html#sphx-glr-gallery-mplot3d-custom-shaded-3d-surface-py
from matplotlib import cbook
from matplotlib import cm
from matplotlib.colors import LightSource
import matplotlib.pyplot as plt
import numpy as np

z = z_array
nrows, ncols = z.shape
print(nrows, ncols)
x = np.linspace(0, 5, ncols)
y = np.linspace(0, 5, nrows)
x, y = np.meshgrid(x, y)

fig, ax = plt.subplots(subplot_kw=dict(projection='3d'))
surf = ax.plot_surface(x, y, z)

plt.show()